In [1]:
#To import all required packages for HW2
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
from fishervector import FisherVectorGMM
%matplotlib inline

In [2]:
#Step(1) to extract images from dataset


# images list to store extracted images from data set folder
images = [] 
# To initilzie the number of images we want to use 
numberOfImages = 35

#getImages() function to extract images from local folder 
def getImages(size):
    
    
    #To loop through images in the dataset folder
    for image in glob("dataset/*.jpg"):
        if (len(images)<size):
            #To read imag from dataset folder using cv2, cv2 reads images in BGR format
            imgBGR = cv2.imread(image)
            #To convert image into gray scale
            grayImage = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2GRAY)
            #To save image in the images list to be used later by another functions
            images.append(grayImage)
    #To show the number of extracted images    
    print (len(images), " Images were extracted from dataset.")
    
    
    
#To call getImages() function   
getImages(numberOfImages)

35  Images were extracted from dataset.


In [3]:
#Step(2): To build a function that computes HoG, Dense SIFT, and Sift.

#A list to store computed image features using HoG
imageFeatures_HoG = []

#A list to store computed image features using Dense SIFT
imageFeatures_Dense = []

#A list to store computed image features using SIFT
imageFeatures_SIFT = []

#Three methods we are going to use to compute image features
methods = ["HoG", "Dense", "SIFT"]




#python function that computes n x d features by calling  HoG, DSIFT and SIFT functions
def computeImageFeatures(image, method):
    
    #To compute HoG:
    if method == "HoG":
        hog = cv2.HOGDescriptor()
        h = hog.compute(image, None)
        
        #print("HoG is computed!")
        
        return h
    
    #To compute Dense:    
    elif method == "Dense":
        sift = cv2.xfeatures2d.SIFT_create()
        step_size = 5
        keyPoints = [cv2.KeyPoint(x, y, step_size) for y in range(0, image.shape[0], step_size) 
                                    for x in range(0, image.shape[1], step_size)]
        keyPoints, descriptors = sift.compute(image, keyPoints)
        
        #print("Dense is computed!")
        
        return descriptors
    
    
    #To compute SIFT
    elif method =="SIFT":
        sift = cv2.xfeatures2d.SIFT_create()
        keyPoints, descriptors = sift.detectAndCompute(image, None)
        
        #print("SIFT is computed!")
  
        return descriptors

#To call computeImageFeatures and store features in three different lists
for method in methods:
    for image in images:
        if method == "HoG":
            imageFeatures_HoG.append(computeImageFeatures(image, method))
        elif method == "Dense":
            imageFeatures_Dense.append(computeImageFeatures(image, method))
        elif method =="SIFT":
            imageFeatures_SIFT.append(computeImageFeatures(image, method))
            
            
print(len(imageFeatures_HoG))
print(len(imageFeatures_Dense))
print(len(imageFeatures_SIFT))

35
35
35


In [4]:
print(imageFeatures_HoG[0])
print("-----------------------------------")
print(imageFeatures_Dense[0])
print("-----------------------------------")
print(imageFeatures_SIFT[0])
print("-----------------------------------")

[[0.1696406 ]
 [0.30958453]
 [0.3215025 ]
 ...
 [0.00311812]
 [0.00336075]
 [0.00932766]]
-----------------------------------
[[  0.   0.   0. ...   0.   0. 153.]
 [  0.   0.   0. ...   0.   0.  58.]
 [  0.   0.   0. ...   0.   0.  21.]
 ...
 [  6.   8. 176. ...   0.   0.   0.]
 [  0.   0. 188. ...   0.   0.   0.]
 [  0.   0. 202. ...   0.   0.   0.]]
-----------------------------------
[[  0.   1. 138. ...   0.  22.  70.]
 [  0.   0.   0. ...   0.  12.  59.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [ 37.  20.  14. ... 107. 108.   5.]
 [ 52.  13.   0. ...  70.   1.   0.]
 [ 15.  74. 150. ...   0.   0.   0.]]
-----------------------------------


In [8]:
#Step(3): To build two functions that compute VLAD and Fisher Vector models of image features

#A list to store computed fv_GMM using HoG features
fv_GMM_HoG = []

#A list to store computed fv_GMM using Dense SIFT featurs
fv_GMM_Dense = []

#A list to store computed fv_GMM using SIFT features
fv_GMM_SIFT = []

#python function that computes fisher vector model
def getFisherVectorModel(method):
    

    #To compute HoG:
    if method == "HoG":
        kd = [8,16]
        
        fv_gmm = FisherVectorGMM().fit_by_bic(np.array(imageFeatures_HoG), choices_n_kernels=[32,64])
        
        return fv_gmm
    
    #To compute Dense:    
    elif method == "Dense":
        #fv_gmm = FisherVectorGMM(n_kernels=[32,64,96]).fit(imageFeatures_Dense)
        fv_gmm = FisherVectorGMM().fit_by_bic(np.array(imageFeatures_Dense), choices_n_kernels=[32,64,96])

        return fv_gmm
    


    elif method =="SIFT":
        #fv_gmm = FisherVectorGMM(n_kernels=[32,64,96]).fit(imageFeatures_SIFT)
        fv_gmm = FisherVectorGMM().fit_by_bic(np.array(imageFeatures_SIFT), choices_n_kernels=[32,64,96])

        
        return fv_gmm


#To call getFisherVectorModel and store features in three different lists
for method in methods:
    for image in images:
        if method == "HoG":
            fv_GMM_HoG.append(getFisherVectorModel(method))
        elif method == "Dense":
            fv_GMM_Dense.append(getFisherVectorModel(method))
        elif method =="SIFT":
            fv_GMM_SIFT.append(getFisherVectorModel(method))

            
print(len(fv_GMM_HoG))
print(len(fv_GMM_Dense))
print(len(fv_GMM_SIFT))

AssertionError: X must be an ndarray with 3 or 4 dimensions